## Analysis of My Alerts

In [1]:
import pandas as pd
import re
import string
import warnings

from bokeh.io import output_notebook
from bokeh.layouts import gridplot
from bokeh.models import HoverTool, ColumnDataSource, FuncTickFormatter
from bokeh.palettes import Category10, gray
from bokeh.plotting import Figure, show, figure
from datetime import datetime, timedelta
from sklearn import tree

In [5]:
# load data
alert_data = pd.read_csv('../data/my_mta_data_for_analysis.csv', encoding='latin1', parse_dates =['time'])

In [6]:
# load visualization tools & settings
#output_notebook()
warnings.filterwarnings('ignore')

#### Total Count of Disruption Notices over Three Years

In [7]:
# load general variables
subway_lines = 'ABCDEFGJLMNQRSWZ1234567'
subway_colors = {
    'A': '#0039A6',
    'B': '#FF6319',
    'C': '#0039A6',
    'D': '#FF6319',
    'E': '#0039A6',
    'F': '#FF6319',
    'G': '#6CBE45',
    'J': '#996633',
    'L': '#A7A9AC',
    'M': '#FF6319',
    'N': '#FCCC0A',
    'Q': '#FCCC0A',
    'R': '#FCCC0A',
    'S': '#808183',
    'Z': '#996633',
    'W': '#FCCC0A',
    '1': '#EE352E',
    '2': '#EE352E',
    '3': '#EE352E',
    '4': '#00933C',
    '5': '#00933C',
    '6': '#00933C',
    '7': '#B933AD'
}

For an initial analysis of when the trains are bad, we're going to remove a few classes of alerts:
- 'update' messages on existing disruptions: we will come back to these
- 'end disruption' announcements
- 'planned services change' announcements, since those are announced ahead of the disruption
- 'retraction' announcements don't really matter to this analysis
- 'non service' announcements don't really matter to this analysis

In [8]:
# adding back end disruption
alert_data_ex = alert_data[
    (~alert_data['estimated'].isin(['planned service change','non service','retraction'])) 
    & (~alert_data['update'])
]

In [9]:
# Find subway lines mentioned
regex = re.compile('[%s]' % re.escape(string.punctuation))
def extract_subway_lines(txt, subway_lines=subway_lines):
    cleaned_txt = regex.sub(' ', txt)
    split_txt = cleaned_txt.split()
    if 'All' in split_txt:
        return [s for s in subway_lines]
    else:
        return [s for s in split_txt if s in subway_lines]
    
alert_data['lines'] = alert_data['title'].apply(
    lambda x: extract_subway_lines(x)
)

In [10]:
alert_data_ex['estimated'].value_counts()

equipment problem           5203
signal                      4602
sick customer               3253
police                      1686
maintenance                 1674
end disruption               988
fire                         830
test                         327
weather                      268
unauthorized person          211
accident                      57
flooding                      53
other agencies                52
event                         47
traffic                       22
construction                  16
high volume                   13
unplanned service change      10
other                          1
Name: estimated, dtype: int64

In [11]:
# take a peak at alerts with updates
alert_data_w_updates = alert_data[
    (~alert_data['estimated'].isin(['end disruption','planned service change','non service','retraction'])) 
]

alert_data_w_updates['estimated'].value_counts()

signal                      12831
equipment problem            8939
sick customer                4365
maintenance                  3586
police                       2615
fire                         1505
weather                      1485
update                       1084
unauthorized person           514
test                          373
accident                      200
construction                  159
unplanned service change       73
flooding                       53
other agencies                 52
event                          47
traffic                        22
high volume                    13
other                           1
Name: estimated, dtype: int64

#### Converted to Daily Averages

In [12]:
print('Average Total Disruptions per Day: ', str(round(len(alert_data_ex)/(3*365),1)))
print()
print(round(alert_data_ex['estimated'].value_counts()/ (3 * 365),1))


Average Total Disruptions per Day:  17.6

equipment problem           4.8
signal                      4.2
sick customer               3.0
police                      1.5
maintenance                 1.5
end disruption              0.9
fire                        0.8
test                        0.3
weather                     0.2
unauthorized person         0.2
accident                    0.1
flooding                    0.0
other agencies              0.0
event                       0.0
traffic                     0.0
construction                0.0
high volume                 0.0
unplanned service change    0.0
other                       0.0
Name: estimated, dtype: float64


#### Messages sent per disruption

In [13]:
msg_per_disruption = pd.DataFrame(
    alert_data[
        (~alert_data['estimated'].isin(['end disruption','planned service change','non service','retraction']))
    ]['estimated'].value_counts() / alert_data_ex['estimated'].value_counts()
)
msg_per_disruption.sort_values('estimated', ascending=False)


,estimated
construction,9.937500
unplanned service change,7.300000
weather,5.541045
accident,3.508772
signal,2.788136
unauthorized person,2.436019
maintenance,2.142174
fire,1.813253
equipment problem,1.718047
police,1.551008


In [14]:
# wow construction is bad: let's take a look
alert_data_ex[
    alert_data_ex['estimated']=='construction'
][['title', 'msg']]

,title,msg
18578,Customer Service Center,"NYCT Customer Service Center is closed Fri., ..."
18579,Customer Service Center,"NYCT Customer Service Center is closed Fri., ..."
18657,"BKLYN, 3 train shuttle Bus, NYPD activity & Ro...","b/d, #3 train shuttle buses are detoured, du..."
28542,"MANH, 1 Trains, Crane collapse",b/d 1 trains are bypassing Franklin St & Chamb...
30937,"QNS, 7 Shuttle Buses, Crane Operation","b/d #7 Shuttle buses are detoured, due to a c..."
34201,L Line Customers/Canarsie Tunnel Reconstructio...,Thank you for signing up to receive periodic e...
40231,"MANH, D Shuttle Buses, Paving",The [D] Shuttle [SB] bus stop on on Chrystie S...
42120,"BKLYN, 3 Shuttle Bus, Construction Equipment","Due to construction equipment on Junius St, [..."
42547,"BKLYN, 3 Train Shuttle Buses, Construction",Pennsylvania Av bound 3 train shuttle buses ar...
50026,"BX, 1 Trains, DOT Work","[1] shuttle buses are detoured, due to DOT wo..."


#### Total Alerts over time

In [15]:
daily_alerts = pd.DataFrame(
    alert_data_ex['time'].apply(
        lambda x: x.date()
    ).value_counts())

daily_alerts.sort_index(inplace=True)
daily_alerts.columns = ['alerts']
daily_alerts['rolling average 7'] = daily_alerts['alerts'].rolling(7).mean()

hover = HoverTool(
    tooltips=[
        ("Date", "$x{%F}"),
        ("Alerts", "$y{0f}")
    ],
    formatters={"$x": "datetime"},
    mode="mouse"
)

plot = Figure(x_axis_type="datetime", plot_width=900, tools=[hover])
plot.multi_line(
    xs=[daily_alerts.index]*2, 
    ys=[daily_alerts['alerts'],daily_alerts['rolling average 7']],
    line_color= ['#aaaaaa','#000000'],
    line_width=[1,3]
)

show(plot)

In [16]:
daily_alerts.sort_values(by='alerts', ascending=False).head(20)

,alerts,rolling average 7
2017-02-09,55,30.142857
2016-01-23,55,24.428571
2015-02-02,46,22.714286
2015-03-05,46,23.142857
2017-01-25,40,23.571429
2017-04-04,40,24.428571
2017-03-15,38,23.571429
2017-01-23,38,23.428571
2017-02-08,37,26.428571
2017-05-18,37,22.142857


##### Notes on some outliers:
-  spike on 2016-01-23, 2016-01-24: [blizzard](https://en.wikipedia.org/wiki/January_2016_United_States_blizzard) big enough to have its own wikipedia entry
- big winter storms on 2015-02-02, 2015-03-05 also caused major disruptions,


In [17]:
# let's exclude bad weather from the analysis: we'll predict that later
daily_alerts = pd.DataFrame(
    alert_data_ex[alert_data_ex['estimated'] != 'weather']['time'].apply(
        lambda x: x.date()
    ).value_counts())

daily_alerts.sort_index(inplace=True)
daily_alerts.columns = ['alerts']
daily_alerts['rolling average 28'] = daily_alerts['alerts'].rolling(21).mean()

plot = Figure(x_axis_type="datetime", plot_width=900, tools=[hover])
plot.multi_line(
    xs=[daily_alerts.index]*2, 
    ys=[daily_alerts['alerts'],daily_alerts['rolling average 28']],
    line_color= ['#aaaaaa','#000000'],
    line_width=[1,3]
)

show(plot)

In [17]:
# worst days this year, ex Weather
daily_alerts[daily_alerts.index > datetime(2016,12,31).date()].sort_values(by='alerts', ascending=False).head(20)

,alerts,rolling average 28
2017-02-09,43,20.523810
2017-04-04,36,19.380952
2017-01-25,36,20.380952
2017-08-18,33,19.142857
2017-05-19,32,19.571429
2017-05-22,31,19.380952
2017-01-23,31,19.809524
2017-03-27,31,17.047619
2017-06-13,31,19.142857
2017-06-29,30,21.190476


The worst day this summer was 2017-08-18: let's see what happened

In [18]:
DATE_TO_MATCH = datetime(2017,8,18).date()

alert_data_ex[['title','msg','time']][
    alert_data_ex['time'].apply(lambda x: x.date()) == DATE_TO_MATCH
]


,title,msg,time
61469,"QNS, F Trains, Mechanical Problems",Due to a train with mechanical problems at Woo...,2017-08-18 00:57:00
61471,"MANH, 1 and 2 Trains, Track Maintenance",Due to track maintenance at 66 St-Lincoln Cent...,2017-08-18 02:47:00
61474,"BX, 2 Trains, Customer Struck by a train",Due to a customer struck by a train at Simpson...,2017-08-18 04:14:00
61475,"BX, 2 Trains, Customer Struck by Trains","Customers on the 2 trains in the Bronx, remai...",2017-08-18 04:20:00
61476,"BX, 2 Trains, Customer Struck by Train",Update on Simpson St 2 Line: NYPD /FDNY on-sce...,2017-08-18 04:26:00
61477,"MANH, F Trains, Track Maintenance",Due to a track maintenance at West 4 St-Washin...,2017-08-18 04:56:00
61482,"QNS, E Trains, Switch Problems",Due to switch problems at Jamaica Center-Parso...,2017-08-18 06:57:00
61485,"QNS, E & F Trains, Unruly Customer",Due to an unruly customer at Jackson Hts-Roose...,2017-08-18 08:17:00
61486,"BX, 4 Train, Mechanical Problems",Due to a train with mechanical problems at Kin...,2017-08-18 08:18:00
61489,"QNS, F Train, Signal Problems","Due to signal problems at Sutphin Blvd, south...",2017-08-18 08:50:00


#### But when should I avoid the subway? This doesn't help. 

Let's see if we can start identifying times that the subway is particularly bad

(note: python & ISO use Monday as first day of week by convention)

In [19]:
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

alert_data['weekday'] = alert_data['time'].apply(lambda x: weekdays[x.weekday()])
alert_data['time_of_day'] = alert_data['time'].apply(lambda x: x.time())
alert_data['date'] = alert_data['time'].apply(lambda x: x.date())

# def bucket_minutes(time, bucket_size = 5):
#     minutes_to_subtract = time.minute % bucket_size
#     datetime_object = datetime(2017,1,1, time.hour, time.minute - minutes_to_subtract)
#     return datetime_object

# hr bucket to start
alert_data['time_bucket'] = alert_data['time'].apply(
    lambda x: ('0' + str(x.hour))[-2:]
)

In [170]:
alert_data_ex_end_and_weather = alert_data[
    (alert_data['estimated'] != 'end disruption') & 
    (alert_data['estimated'] != 'planned service change') &
    (alert_data['estimated'] != 'weather') &
    (~alert_data['update'])
]

In [171]:
weekly_heatmap_data = alert_data_ex_end_and_weather[
    ['weekday', 'time_bucket', 'count']
].groupby(
    ['weekday', 'time_bucket']
).count().reset_index()

weekly_heatmap_data['alpha'] = weekly_heatmap_data['count'] / max(weekly_heatmap_data['count'])

In [172]:
p = figure(
    title='Heat Map of Disruptions',
    x_axis_location="above",
    x_range = weekdays,
    y_range = (23.5,-0.5),
    tools='hover'
)

p.plot_height = 800
p.plot_width = 800
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None

p.rect(
    source=weekly_heatmap_data,
    x='weekday',
    y='time_bucket',
    width= 0.99,
    height = 0.95,
    color='#000000',
    alpha='alpha'
)

p.select_one(HoverTool).tooltips = [
    ('time', '@time_bucket'),
    ('disruptions', '@count')
]

show(p)

In [173]:
# likelihood of a disruption happening, on any particular hour of a day

# get weekdays in sample

START_DATE = datetime(2014,9,1).date()
END_DATE = datetime(2017,8,30).date()

this_day = START_DATE
weekdays_count = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0}

while this_day <= END_DATE:
    weekdays_count[this_day.weekday()] += 1
    this_day = this_day + timedelta(days=1)

# get unique day/hour counts
_weekday_hrly_list = alert_data_ex_end_and_weather[
    ['weekday', 'date', 'time_bucket', 'estimated']
].groupby([ 'weekday', 'time_bucket', 'date']).count().reset_index()

_weekday_hrly_cnt = _weekday_hrly_list[
    [ 'weekday', 'time_bucket', 'date']
].groupby(['weekday', 'time_bucket']).count().reset_index()

_weekday_hrly_cnt['probability'] = _weekday_hrly_cnt[
    ['weekday','date']
].apply(lambda x: x[1]/157 if x[0] in ['Monday', 'Tuesday', 'Wednesday'] else x[1]/156, axis=1)

In [174]:
# daily facet plot

_weekday_hrly_cnt.head()

plot_list = []
for i, d in enumerate(weekdays):
    _plot = figure(plot_width=300, plot_height=200, y_range=(0,1), title=d, y_minor_ticks=2)
    _plot.vbar(
        x=[x for x in range(24)], 
        top=_weekday_hrly_cnt['probability'][_weekday_hrly_cnt['weekday'] == d],
        width= 1,
        color=Category10[7][i]
    )
    plot_list.append(_plot)
    
grid = gridplot(plot_list, ncols=3, plot_width=300, plot_height=200)
show(grid)

In [25]:
# try 15m to graph by day
alert_data['time_bucket_15'] = alert_data['time'].apply(
    lambda x: x.hour + int(x.minute/15)/4
)

alert_data_ex_end_and_weather = alert_data[
    (alert_data['estimated'] != 'end disruption') & 
    (alert_data['estimated'] != 'planned service change') &
    (alert_data['estimated'] != 'weather') & 
    (~alert_data['update']) 
]

weekly_linechart_data = alert_data_ex_end_and_weather[
    ['weekday', 'time_bucket_15', 'count']
].groupby(
    ['weekday', 'time_bucket_15']
).count().reset_index()


In [26]:
# daily facet plot
plot_list = []
for i, d in enumerate(weekdays):
    _plot = figure(plot_width=300, plot_height=200, y_range=(0,80), title=d, y_minor_ticks=2)
    _plot.line(
        x=[x/4 for x in range(24*4)], 
        y=weekly_linechart_data['count'][weekly_linechart_data['weekday'] == d],
        color=Category10[7][i]
    )
    plot_list.append(_plot)
    
grid = gridplot(plot_list, ncols=3, plot_width=300, plot_height=200)
show(grid)

### Let's start looking at the numbers by Subway Line

In [175]:
# get raw_data_by_line and weekday_data_by_line
raw_data_by_line = {}
weekday_data_by_line = {}

for line in subway_lines:
    raw_data_by_line[line] = alert_data_ex_end_and_weather[
        alert_data_ex_end_and_weather['lines'].apply(
            lambda x: line in x
        )
    ]
    # get unique day/hour counts
    _weekday_hrly_list = raw_data_by_line[line][
        ['weekday', 'date', 'time_bucket', 'estimated']
    ].groupby([ 'weekday', 'time_bucket', 'date']).count().reset_index()
    
    _weekday_hrly_cnt = _weekday_hrly_list[
        [ 'weekday', 'time_bucket', 'date']
    ].groupby(['weekday', 'time_bucket']).count().reset_index()
    
    _weekday_hrly_cnt['probability'] = _weekday_hrly_cnt[
        ['weekday','date']
    ].apply(lambda x: x[1]/157 if x[0] in ['Monday', 'Tuesday', 'Wednesday'] else x[1]/156, axis=1)
    
    weekday_data_by_line[line] = _weekday_hrly_cnt

In [176]:
# daily facet plot alerts ex updates
this_line = 'W'

plot_list = []
for i, d in enumerate(weekdays):
    _plot = figure(plot_width=300, plot_height=200, y_range=(0,.5), title=d, y_minor_ticks=2)
    _plot.vbar(
        x=[x for x in range(24)], 
        top=weekday_data_by_line[this_line]['probability'][weekday_data_by_line[this_line]['weekday'] == d],
        width= 1,
        color=subway_colors[this_line]
    )
    plot_list.append(_plot)
    
grid = gridplot(plot_list, ncols=3, plot_width=300, plot_height=200)
show(grid)

In [30]:
types_of_disruptions = alert_data_ex_end_and_weather['estimated'].unique()
types_of_disruptions

array(['equipment problem', 'sick customer', 'signal', 'event', 'police',
       'maintenance', 'fire', 'test', 'retraction', 'high volume',
       'flooding', 'traffic', 'other agencies', 'non service', 'accident',
       'construction', 'unplanned service change', 'unauthorized person',
       'other'], dtype=object)

In [31]:
# when does each type of issue happen?
raw_data_by_type = {}
weekday_data_by_type = {}

for disruption in types_of_disruptions:
    raw_data_by_type[disruption] = alert_data_ex_end_and_weather[
        alert_data_ex_end_and_weather['estimated'] == disruption
    ]
    # get unique day/hour counts
    _weekday_hrly_list = raw_data_by_type[disruption][
        ['weekday', 'date', 'time_bucket', 'estimated']
    ].groupby([ 'weekday', 'time_bucket', 'date']).count().reset_index()
    
    _weekday_hrly_cnt = _weekday_hrly_list[
        [ 'weekday', 'time_bucket', 'date']
    ].groupby(['weekday', 'time_bucket']).count().reset_index()
    
    _weekday_hrly_cnt['probability'] = _weekday_hrly_cnt[
        ['weekday','date']
    ].apply(lambda x: x[1]/157 if x[0] in ['Monday', 'Tuesday', 'Wednesday'] else x[1]/156, axis=1)
    
    weekday_data_by_type[disruption] = _weekday_hrly_cnt


In [34]:
# daily facet plot alerts ex updates
this_disruption = 'equipment problem'

plot_list = []
for i, d in enumerate(weekdays):
    _plot = figure(plot_width=300, plot_height=200, y_range=(0,.5), title=d, y_minor_ticks=2)
    _plot.vbar(
        x=[x for x in range(24)], 
        top=weekday_data_by_type[this_disruption]['probability'][weekday_data_by_type[this_disruption]['weekday'] == d],
        width= 1,
        color='#888888'
    )
    plot_list.append(_plot)
    
grid = gridplot(plot_list, ncols=3, plot_width=300, plot_height=200)
show(grid)

Some Notes from looking at each of these:
- most problems follow the same pattern: even equipment problems peak at peak times
- worst in the morning, worse earlier in the week
- afternoon sick customers get more prevalent later in the week, by friday its as many as AM (drinking
- Signal problems have a big spike on Monday AM, around 6
- *huge* signal spike on Monday 6AM
- events are rare, most common on fridays
- police activity higher later in the week, fairly commone in afternoon
- maintenance happens exactly off hours: 0-5 & 10-15
- fire is faily constant during the day and week

In [82]:
# disruption by line heatmap
disruption_by_line = []

def generate_disruption_list(disruption, lines, disruption_by_line= disruption_by_line):
    for line in lines:
        disruption_by_line.append({"line": line, "disruption": disruption, "count": 1})
    return None
    

alert_data_ex_end_and_weather[['estimated', 'lines']].apply(
    lambda x: generate_disruption_list(x[0],x[1]), axis=1
)

disruptions_df = pd.DataFrame(disruption_by_line).groupby(['disruption', 'line']).count().reset_index()
disruptions_df['perc_max'] = disruptions_df['count'] / max( disruptions_df['count'])
disruptions_df['tmp'] = disruptions_df['line'].apply(lambda x: subway_lines.find(x))

In [103]:
p = figure(
    title='Heat Map of Disruptions',
    x_axis_location="above",
    x_range = (-0.5,21.5),
    y_range = [t for t in types_of_disruptions],
    tools='hover'
)

p.plot_height = 19 * 40
p.plot_width = 22 * 40
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
plot.xaxis[0].ticker.desired_num_ticks = 22

p.rect(
    source=disruptions_df,
    x='tmp',
    y='disruption',
    width= 0.95,
    height = 0.95,
    color='#000000',
    alpha='perc_max'
)

p.select_one(HoverTool).tooltips = [
    ('line', '@line'),
    ('disruptions', '@count')
]

p.xaxis.ticker = list(range(22))
p.xaxis.major_label_overrides = {
    0:'A',
    1:'B',
    2:'C',
    3:'D',
    4:'E',
    5:'F',
    6:'G',
    7:'J',
    8:'L',
    9:'M',
    10:'N',
    11:'Q',
    12:'R',
    13:'S',
    14:'Z',
    15:'1',
    16:'2',
    17:'3',
    18:'4',
    19:'5',
    20:'6',
    21:'7'
}

show(p)

In [110]:
# another approach: lets look at the distribution of # alerts by day

count_of_alerts = pd.DataFrame(daily_alerts['alerts'].value_counts().reset_index())

p = figure(title='Daily System Alerts')

p.vbar(top = count_of_alerts['alerts'], x = count_of_alerts['index'], width = 1)
show(p)

In [152]:
daily_alerts_expand = daily_alerts.reset_index()
daily_alerts_expand['weekday'] = daily_alerts_expand['index'].apply(lambda x: x.weekday())
daily_alerts_expand['sample_yr'] = daily_alerts_expand['index'].apply(lambda x: (x + timedelta(days=122)).year)

In [153]:
weekday_alerts = daily_alerts_expand[
    ['weekday', 'alerts', 'index']
].groupby(['weekday', 'alerts']).count().reset_index()

plot_list = []
for i, d in enumerate(weekdays):
    _plot = figure(plot_width=300, plot_height=200, y_range=(0,40), title=d, y_minor_ticks=2)
    _plot.vbar(
        x=weekday_alerts['alerts'][weekday_alerts['weekday'] == i] ,
        top=weekday_alerts['index'][weekday_alerts['weekday'] == i] ,
        width= 1,
        color='#888888'
    )
    plot_list.append(_plot)
    
grid = gridplot(plot_list, ncols=3, plot_width=300, plot_height=200)
show(grid)

In [157]:
# look at how it's changed by year
yearly_alerts = daily_alerts_expand[
    ['sample_yr', 'alerts', 'index']
].groupby(['sample_yr', 'alerts']).count().reset_index()

#yearly_alerts['color'] = yearly_alerts['sample_yr'].apply(lambda x: Category10[4][x])

plot_list = []
for i in (2015, 2016, 2017):
    _plot = figure(plot_width=500, plot_height=200, y_range=(0,40), x_range=(0,45), title=str(i), y_minor_ticks=2)
    _plot.vbar(
        x=yearly_alerts['alerts'][yearly_alerts['sample_yr'] == i] ,
        top=yearly_alerts['index'][yearly_alerts['sample_yr'] == i] ,
        width= 1,
        color='#888888'
    )
    plot_list.append(_plot)
    
grid = gridplot(plot_list, ncols=1, plot_width=500, plot_height=200)
show(grid)
